In [4]:
# Lets first import the libraries we will need 
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pandas as pd
import numpy as np

In [4]:
# Second step will be importing the data to our Jupyter Notebook
# The data I used to build this project can be found here:
# https://www.theguardian.com/sport/datablog/2012/jun/25/olympic-medal-winner-list-data#data

filepath = #

olympics = pd.read_csv(filepath, 
    skiprows = [0,1,2], header = 1)

In [4]:
# After the first look at the database I decided to use the columns MEDAL, NOC, and EDITION
# To build graphics using Bokeh server

data = olympics.loc[:, ['Medal', 'NOC', 'Edition']]
data.sort_values(by = ['NOC', 'Edition'], inplace = True)

In [4]:
# The following step was creating these functions to count how many GOLD, SILVER, and BRONZE medals 
# Each country got over the years (from 1896 to 2008)

def separete_gold(value):
    if value == 'Gold':
        return 1
    else: 
        return 0
    
def separete_silver(value):
    if value == 'Silver':
        return 1
    else: 
        return 0

def separete_bronze(value):
    if value == 'Bronze':
        return 1
    else: 
        return 0

# Adding new columns to our database

data['Gold'] = data.Medal.apply(separete_gold)
data['Silver'] = data.Medal.apply(separete_silver)
data['Bronze'] = data.Medal.apply(separete_bronze)

In [4]:
# Here we rearange the data so that we can see how the medals are distributed over 
# The years. Here we make sure that every NOC will be represented every 4 years and
# We get the correct corresponding YEAR - NOC - GOLD, SILVER, BRONZE

lista_anos = list(data['Edition'].unique())
lista_noc = list(data['NOC'].unique())
lista_plot = []

lista_noc_anos = []
for noc in lista_noc:
    for year in lista_anos:
        lista_aux = list(data[(data['NOC'] == noc) 
                & (data['Edition'] == year)][['Gold', 'Silver', 'Bronze']].sum().values)
        lista_aux.extend([year, noc])
        lista_noc_anos.append(lista_aux)

In [4]:
# Finally we make a new DataFrame using the data we reorganized
data_strutured = np.array(lista_noc_anos)
dataframe = pd.DataFrame(data_strutured, columns=['Gold',
                                                 'Silver',
                                                 'Bronze',
                                                 'Year',
                                                 'NOC'])


dataframe.Gold = np.array(dataframe.Gold, dtype=np.int32)
dataframe.Silver = np.array(dataframe.Silver, dtype=np.int32)
dataframe.Bronze = np.array(dataframe.Bronze, dtype=np.int32)
dataframe.Year = np.array(dataframe.Year, dtype=np.int32)

In [4]:
# Here we start to prepare our data to be used to draw the graphics
source = ColumnDataSource(data = {
    'x': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Year'],
    'y': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Gold'],
})

# Set the limits for x_axis
xmin, xmax = min(dataframe['Year']), max(dataframe['Year'])

# Creating a figure and writing a callback
plot = figure(x_axis_label='Year', 
              y_axis_label = 'Total Medals',
             x_range= (xmin - 8, xmax + 8))

def update_medals(attr, old, new):
    medal = menu_medals.value
    noc = menu_noc.value
    new_data ={
        'x': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Year'],
        'y': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')[medal],
    }
    source.data = new_data
    y_axis_label = 'Total Medals'

# Creating menus and adding on_change methods
menu_medals = Select(options=['Gold','Silver', 'Bronze'],
value = 'Gold',title ='Medals')

menu_noc = Select(options=lista_noc,
value = 'USA',title ='Countries')

menu_medals.on_change('value', update_medals)
menu_noc.on_change('value', update_medals)

# Finally plotting and organazing everything in a layout
plot.line(x = 'x', y = 'y', source = source)

layout = column(row(menu_medals, menu_noc), plot)

curdoc().add_root(layout)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3588 entries, 0 to 3587
Data columns (total 5 columns):
Gold      3588 non-null int32
Silver    3588 non-null int32
Bronze    3588 non-null int32
Year      3588 non-null int32
NOC       3588 non-null object
dtypes: int32(4), object(1)
memory usage: 84.2+ KB
None
